<a href="https://colab.research.google.com/github/dhallankit/INST767_MinHash_LSH_Assignment/blob/master/INST767_A2_Ankit_Dhall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INST 767 - Assignment 2 - MinHash Implementation (Jaccard Similarity) Using PySpark

### Installing packages

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop3.2.tgz
!tar -xvf spark-3.0.0-preview2-bin-hadoop3.2.tgz
!pip install -q findspark

spark-3.0.0-preview2-bin-hadoop3.2/
spark-3.0.0-preview2-bin-hadoop3.2/data/
spark-3.0.0-preview2-bin-hadoop3.2/data/streaming/
spark-3.0.0-preview2-bin-hadoop3.2/data/streaming/AFINN-111.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_binary_classification_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_kmeans_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_multiclass_classification_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_lda_libsvm_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/iris_libsvm.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/pagerank_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_linear_regression_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/pic_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/als/
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/als/test.data
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/als/sample_movielens_rati

### Setting environment variables

In [0]:
import os
from os import walk
import glob
import re
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop3.2"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.sql.functions import col
import sys

## Choose zip file containing sample documents (expected .txt documents in zip file)

In [0]:
#Uploaded files that need to be compared with sample file
from google.colab import files

uploaded = files.upload()


Saving Transcripts.zip to Transcripts.zip


### Unzipping Archive file

In [14]:
!unzip Transcripts.zip

Archive:  Transcripts.zip
   creating: Transcripts/
   creating: Transcripts/DualTerms/
  inflating: Transcripts/DualTerms/Bush2001.txt  
  inflating: Transcripts/DualTerms/Bush2005.txt  
  inflating: Transcripts/DualTerms/Obama2009.txt  
 extracting: Transcripts/DualTerms/Obama2013.txt  
  inflating: Transcripts/Trump2017.txt  


In [15]:
!ls
!pwd

sample_data				Transcripts
spark-3.0.0-preview2-bin-hadoop3.2	Transcripts.zip
spark-3.0.0-preview2-bin-hadoop3.2.tgz
/content


In [16]:
givenfiles = [file for file in glob.glob("Transcripts/**/*.txt", recursive=True)]
givenfiles

['Transcripts/Trump2017.txt',
 'Transcripts/DualTerms/Obama2013.txt',
 'Transcripts/DualTerms/Bush2005.txt',
 'Transcripts/DualTerms/Obama2009.txt',
 'Transcripts/DualTerms/Bush2001.txt']

## Choose sample file to compate other documents with

In [17]:
# Upload sample file for other files to be matches against
sample = files.upload()

Saving Washington1789.txt to Washington1789.txt


## Appending text files into separate continuous strings

In [26]:
doclist = []
docstringlist = []

for f in givenfiles:
  workfile = open(f, 'r')
  workdoc = []
  for line in workfile:
    line = re.sub("[!?@,.;:]", '', line)
    workdoc.append(line.strip().lower())
  workdoc = [i for i in workdoc if i]
  doclist.append(workdoc)

doclist=[i for i in doclist if i != []]

for lis in doclist:
  # print(lis)
  teststring = ""
  for i in lis:
    # print(teststring)
    teststring+=i+" "
  docstringlist.append(teststring)

docstringlist

["chief justice roberts president carter president clinton president bush president obama fellow americans and people of the world thank you we the citizens of america are now joined in a great national effort to rebuild our country and to restore its promise for all of our people together we will determine the course of america and the world for years to come we will face challenges we will confront hardships but we will get the job done every four years we gather on these steps to carry out the orderly and peaceful transfer of power and we are grateful to president obama and first lady michelle obama for their gracious aid throughout this transition they have been magnificent today’s ceremony however has very special meaning because today we are not merely transferring power from one administration to another or from one party to another – but we are transferring power from washington dc and giving it back to you the american people for too long a small group in our nation’s capital 

## Appending sample document into a single continuous string

In [72]:
sampledoclist=[]
for f in sample:
  workfile = open(f, 'r')
  workdoc = []
  for line in workfile:
    line = re.sub("[!?@,.;:]", '', line)
    workdoc.append(line.strip().lower())
  workdoc = [i for i in workdoc if i]
  sampledoclist.append(workdoc)

sampledoclist
sampledocstring = ""
separator = " "
for i in sampledoclist:
  sampledocstring = " ".join(i)
sampledocstring

'fellow citizens of the senate and the house of representatives among the vicissitudes incident to life no event could have filled me with greater anxieties than that of which the notification was transmitted by your order and received on the fourteenth day of the present month on the one hand i was summoned by my country whose voice i can never hear but with veneration and love from a retreat which i had chosen with the fondest predilection and in my flattering hopes with an immutable decision as the asylum of my declining years a retreat which was rendered every day more necessary as well as more dear to me by the addition of habit to inclination and of frequent interruptions in my health to the gradual waste committed on it by time on the other hand the magnitude and difficulty of the trust to which the voice of my country called me being sufficient to awaken in the wisest and most experienced of her citizens a distrustful scrutiny into his qualifications could not but overwhelm wit

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, NGram, HashingTF, MinHashLSH

In [59]:
sampledoc = spark.createDataFrame(sampledoclist[0], "string").toDF("text")
sampledoc.show()

+--------------------+
|                text|
+--------------------+
|fellow citizens o...|
|among the vicissi...|
|such being the im...|
|by the article es...|
|i dwell on this p...|
|besides the ordin...|
|to the preceeding...|
|having thus impar...|
+--------------------+



In [23]:
querydoc = []
for i in doclist:
  querydoc.append(spark.createDataFrame(i, "string").toDF("text"))

querydoc[0].show()

+--------------------+
|                text|
+--------------------+
|chief justice rob...|
|we the citizens o...|
|together we will ...|
|we will face chal...|
|every four years ...|
|today’s ceremony ...|
|for too long a sm...|
|washington flouri...|
|politicians prosp...|
|the establishment...|
|their victories h...|
|that all changes ...|
|it belongs to eve...|
|this is your day ...|
|and this the unit...|
|what truly matter...|
|january 20th 2017...|
|the forgotten men...|
|everyone is liste...|
|you came by the t...|
+--------------------+
only showing top 20 rows



## Appeding sample document string in a list at position 0 and adding test document strings ahead in the list

In [99]:
finalstringlist = []
finalstringlist.append(sampledocstring)


for string in docstringlist:
  finalstringlist.append(string)

finalstringlist

# finallistdf = spark.createDataFrame(finalstringlist, "string").toDF("text")
# finallistdf.show()

['fellow citizens of the senate and the house of representatives among the vicissitudes incident to life no event could have filled me with greater anxieties than that of which the notification was transmitted by your order and received on the fourteenth day of the present month on the one hand i was summoned by my country whose voice i can never hear but with veneration and love from a retreat which i had chosen with the fondest predilection and in my flattering hopes with an immutable decision as the asylum of my declining years a retreat which was rendered every day more necessary as well as more dear to me by the addition of habit to inclination and of frequent interruptions in my health to the gradual waste committed on it by time on the other hand the magnitude and difficulty of the trust to which the voice of my country called me being sufficient to awaken in the wisest and most experienced of her citizens a distrustful scrutiny into his qualifications could not but overwhelm wi

### Tokenizing the string list to get words and counts

In [165]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
countTokens = udf(lambda words: len(words), IntegerType())

regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")
regexTokenized = regexTokenizer.transform(finallistdf)
regexTokenized.select("text","words").withColumn("tokens", countTokens(col("words"))).show(truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Adding index to the combined tokenized dataframe set

In [187]:
# !pip install pyspark.sql.functions.
from pyspark.sql.functions import monotonically_increasing_id, row_number, desc

# This will return a new DF with all the columns + id
finallist = regexTokenized.toDF("text", "words")
# finallist = finallist.toDF()
finallist.show()
# finallist = finallist.rdd.zipWithUniqueId()
# finallist = finallist.toDF()
# finallist
# finallist = finallist.withColumn('text', finallist['_1'].getItem('text'))
# finallist = finallist.withColumn('words', finallist['_1'].getItem('words'))
# finallist = finallist.withColumn('index', finallist['_2'])
# finallist.select('index', 'text', 'words').show()
# finallist.show()
finallist = finallist.withColumn('index', row_number().over(Window.orderBy(monotonically_increasing_id())) - 1)
finallist.show()


+--------------------+--------------------+
|                text|               words|
+--------------------+--------------------+
|fellow citizens o...|[fellow, citizens...|
|chief justice rob...|[chief, justice, ...|
|vice president ch...|[vice, president,...|
|i stand here toda...|[i, stand, here, ...|
|president clinton...|[president, clint...|
+--------------------+--------------------+

+--------------------+--------------------+-----+
|                text|               words|index|
+--------------------+--------------------+-----+
|fellow citizens o...|[fellow, citizens...|    0|
|chief justice rob...|[chief, justice, ...|    1|
|vice president ch...|[vice, president,...|    2|
|i stand here toda...|[i, stand, here, ...|    3|
|president clinton...|[president, clint...|    4|
+--------------------+--------------------+-----+



## Implementing MinHash package from PySpark library with a Pipeline model

In [0]:
 import pyspark.sql.functions as f
 model = Pipeline(stages=[
        HashingTF(inputCol="words", outputCol="vectors"),
        MinHashLSH(inputCol="vectors", outputCol="lsh", numHashTables=10)
    ]).fit(finallist)

In [0]:
finallist_hashed = model.transform(finallist)
finallist_match = model.stages[-1].approxSimilarityJoin(finallist_hashed, finallist_hashed, 0.9)

# The output of this cell displays top 5 documents that match the original document based on MinHash (Jaccard Distance)

### IndexL shows reference document
### IndexR shows documents being compared with reference document

In [197]:
finallist_match.select(f.col('datasetA.index').alias('IndexL'), f.col('datasetA.words').alias('DocumentonLeftSide'),
                 f.col('datasetB.index').alias('IndexR'), f.col('datasetB.words').alias('DocumentonRightSide'),
                 f.col('distCol').alias('Jaccard Distance')).filter('IndexL = 0').sort(col('Jaccard Distance').asc()).limit(5).show()

+------+--------------------+------+--------------------+------------------+
|IndexL|  DocumentonLeftSide|IndexR| DocumentonRightSide|  Jaccard Distance|
+------+--------------------+------+--------------------+------------------+
|     0|[fellow, citizens...|     0|[fellow, citizens...|               0.0|
|     0|[fellow, citizens...|     4|[president, clint...|0.8398821218074657|
|     0|[fellow, citizens...|     2|[vice, president,...|0.8479020979020979|
|     0|[fellow, citizens...|     3|[i, stand, here, ...| 0.851937984496124|
|     0|[fellow, citizens...|     1|[chief, justice, ...|0.8615847542627884|
+------+--------------------+------+--------------------+------------------+

